In [1]:
# Importing all necessary libraries
import cv2
import os
import shutil

In [2]:
# Read the video from specified path
cam = cv2.VideoCapture("C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/ppl2.mp4")
# frames = cam.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cam.get(cv2.CAP_PROP_FPS)
# seconds = round(frames / fps)
print(fps)

30.0


In [3]:
try:
	
	# creating a folder named data
	if not os.path.exists('frames'):
		os.makedirs('frames')

# if not created then raise error
except OSError:
	print ('Error: Creating directory of data')

# frame
currentframe = 0

while(True):
	
	# reading from frame
	ret,frame = cam.read()

	if ret:
		# if video is still left continue creating images
            if(currentframe<10):name = './frames/0' + '%d.png' % currentframe
            else:name = './frames/' + '%d.png' % currentframe
            print ('Creating...' + name)

		# writing the extracted images
            cv2.imwrite(name, frame)

		# increasing counter so that it will
		# show how many frames are created
            currentframe += 1
	else:
		break
cam.release()
cv2.destroyAllWindows()

Creating..../frames/00.png
Creating..../frames/01.png
Creating..../frames/02.png
Creating..../frames/03.png
Creating..../frames/04.png
Creating..../frames/05.png
Creating..../frames/06.png
Creating..../frames/07.png
Creating..../frames/08.png
Creating..../frames/09.png
Creating..../frames/10.png
Creating..../frames/11.png
Creating..../frames/12.png
Creating..../frames/13.png
Creating..../frames/14.png
Creating..../frames/15.png
Creating..../frames/16.png
Creating..../frames/17.png
Creating..../frames/18.png
Creating..../frames/19.png
Creating..../frames/20.png
Creating..../frames/21.png
Creating..../frames/22.png
Creating..../frames/23.png
Creating..../frames/24.png
Creating..../frames/25.png
Creating..../frames/26.png
Creating..../frames/27.png
Creating..../frames/28.png
Creating..../frames/29.png
Creating..../frames/30.png
Creating..../frames/31.png
Creating..../frames/32.png
Creating..../frames/33.png
Creating..../frames/34.png
Creating..../frames/35.png
Creating..../frames/36.png
C

In [4]:
from PIL import Image
import os
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import random
from math import log
from tqdm import tqdm

In [5]:
def getImageMatrix(imageName):
    im = Image.open(imageName, mode="r", formats=None) 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
        color=0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
                row.append((pix[width,height]))
        image_matrix.append(row)
    return image_matrix,image_size[0],color

In [6]:
def LogisticEncryption(imageName, key):
    N = 256
    key_list = [ord(x) for x in key]
    G = [key_list[0:4] ,key_list[4:8], key_list[8:12]]
    g = []
    R = 1
    for i in range(1,4):
        s = 0
        for j in range(1,5):
            s += G[i-1][j-1] * (10**(-j))
        g.append(s)
        R = (R*s) % 1
    L = (R + key_list[12]/256) % 1
    S_x = round(((g[0]+g[1]+g[2])*(10**4) + L *(10**4)) % 256)
    V1 = sum(key_list)
    V2 = key_list[0]
    for i in range(1,13):
        V2 = V2 ^ key_list[i]
    V = V2/V1
    L_y = (V+key_list[12]/256) % 1
    S_y = round((V+V2+L_y*10**4) % 256)
    C1_0 = S_x
    C2_0 = S_y
    C = round((L*L_y*10**4) % 256)
    C_r = round((L*L_y*10**4) % 256)
    C_g = round((L*L_y*10**4) % 256)
    C_b = round((L*L_y*10**4) % 256)
    x = 4*(S_x)*(1-S_x)
    y = 4*(S_y)*(1-S_y)
    imageMatrix,dimension, color = getImageMatrix(imageName)
    LogisticEncryptionIm = []
    for i in range(dimension):
        row = []
        for j in range(dimension):
            while x <0.8 and x > 0.2 :
                x = 4*x*(1-x)
            while y <0.8 and y > 0.2 :
                y = 4*y*(1-y)
            x_round = round((x*(10**4))%256)
            y_round = round((y*(10**4))%256)
            C1 = x_round ^ ((key_list[0]+x_round) % N) ^ ((C1_0 + key_list[1])%N)
            C2 = x_round ^ ((key_list[2]+y_round) % N) ^ ((C2_0 + key_list[3])%N) 
            if color:
                C_r =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][0]) % N) ^ ((C_r + key_list[7]) % N)
                C_g =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][1]) % N) ^ ((C_g + key_list[7]) % N)
                C_b =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][2]) % N) ^ ((C_b + key_list[7]) % N)
                row.append((C_r,C_g,C_b))
                C = C_r
            else:
                C = ((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j]) % N) ^ ((C + key_list[7]) % N)
                row.append(C)
            x = (x + C/256 + key_list[8]/256 + key_list[9]/256) % 1
            y = (x + C/256 + key_list[8]/256 + key_list[9]/256) % 1
            for ki in range(12):
                key_list[ki] = (key_list[ki] + key_list[12]) % 256
                key_list[12] = key_list[12] ^ key_list[ki]
        LogisticEncryptionIm.append(row)
    im = Image.new('L', (dimension, dimension))
    if color:
        im = Image.new('RGB', (dimension, dimension))
    else: 
        im = Image.new('L', (dimension, dimension)) # L is for Black and white pixels    
    pix = im.load()
    for x in range(dimension):
        for y in range(dimension):
            pix[x, y] = LogisticEncryptionIm[x][y]
    im.save(image+"encryp"+imageName[10:-4]+".bmp", "BMP")

In [7]:
image = "./new"
ext = ".bmp"
key = "abcdefghijklm"
i=0

In [8]:
import shutil as sh
folder_dir = "./frames"
cnt=0
cropped_images = []
for images in os.listdir(folder_dir):
    face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
    img = cv2.imread('./frames/'+images)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 1)
        cv2.imwrite('./new/face detection result.png', img)
    for (x, y, w, h) in faces:
        cropped_image = img[y+1:y+h, x+1:x+w]
        filename=image+"face"+ext
        cv2.imwrite(filename, cropped_image)
        LogisticEncryption(filename, key)
        filename=image+"encryp"+ext
        LogisticResult = cv2.imread(filename,cv2.IMREAD_COLOR)
        img[y+1:y+h, x+1:x+w] = LogisticResult
        cv2.imwrite('./encrypted_frames/EncryptionResult'+str(cnt)+ext, img)
        sh.move('./encrypted_frames/EncryptionResult'+str(cnt)+ext,'./SquareDetectionResults/encrypted_frames/EncryptionResult'+images)
    cnt+=1
        

In [9]:
# importing libraries
import os
import cv2
from PIL import Image
import os.path
import shutil
# Checking the current directory path
print(os.getcwd())

# Folder which contains all the images
# from which video is to be generated

os.chdir("C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/SquareDetectionResults/encrypted_frames")
path = "C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/SquareDetectionResults/encrypted_frames"

mean_height = 0
mean_width = 0
num_of_images = len(os.listdir('.'))
print(num_of_images)

for file in os.listdir('.'):
	im = Image.open(os.path.join(path, file))
	width, height = im.size
	mean_width += width
	mean_height += height
	# im.show() # uncomment this for displaying the image

# Finding the mean height and width of all images.
# This is required because the video frame needs
# to be set with same width and height. Otherwise
# images not equal to that width height will not get
# embedded into the video
mean_width = int(mean_width / num_of_images)
mean_height = int(mean_height / num_of_images)

# print(mean_height)
# print(mean_width)

# Resizing of the images to give
# them same width and height
for file in os.listdir('.'):
	if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith("png"):
		# opening image using PIL Image
		im = Image.open(os.path.join(path, file))

		# im.size includes the height and width of image
		width, height = im.size
		print(width, height)

		# resizing
		imResize = im.resize((mean_width, mean_height), Image.ANTIALIAS)
		imResize.save( file, 'JPEG', quality = 95) # setting quality
		# printing each resized image name
		print(im.filename.split('\\')[-1], " is resized")


# Video Generating function
def generate_video():
	image_folder = "C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/SquareDetectionResults/encrypted_frames" # make sure to use your folder
	video_name = 'encryptedvideo.avi'
	os.chdir("C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/new")
	
	images = [img for img in os.listdir(image_folder)
			if img.endswith(".jpg") or
				img.endswith(".jpeg") or
				img.endswith("png")]
	
	# Array images should only consider
	# the image files ignoring others if any
	print(images)

	frame = cv2.imread(os.path.join(image_folder, images[0]))

	# setting the frame width, height width
	# the width, height of first image
	height, width, layers = frame.shape

	video = cv2.VideoWriter(video_name, 0, fps, (width, height))

	# Appending the images to the video one by one
	for image in images:
		video.write(cv2.imread(os.path.join(image_folder, image)))
	# Deallocating memories taken for window creation
	cv2.destroyAllWindows()
	video.release() # releasing the video generated


# Calling the generate_video function
generate_video()


C:\Users\USER\Downloads\Secure-Sharing-of-CCTV-footage-master (2)\Secure-Sharing-of-CCTV-footage-master
61
1920 1080
EncryptionResult00.png  is resized
1920 1080


C:\Users\USER\AppData\Local\Temp\ipykernel_13724\886580481.py:51: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  imResize = im.resize((mean_width, mean_height), Image.ANTIALIAS)


EncryptionResult01.png  is resized
1920 1080
EncryptionResult02.png  is resized
1920 1080
EncryptionResult03.png  is resized
1920 1080
EncryptionResult04.png  is resized
1920 1080
EncryptionResult05.png  is resized
1920 1080
EncryptionResult06.png  is resized
1920 1080
EncryptionResult07.png  is resized
1920 1080
EncryptionResult08.png  is resized
1920 1080
EncryptionResult09.png  is resized
1920 1080
EncryptionResult10.png  is resized
1920 1080
EncryptionResult11.png  is resized
1920 1080
EncryptionResult12.png  is resized
1920 1080
EncryptionResult13.png  is resized
1920 1080
EncryptionResult14.png  is resized
1920 1080
EncryptionResult15.png  is resized
1920 1080
EncryptionResult16.png  is resized
1920 1080
EncryptionResult17.png  is resized
1920 1080
EncryptionResult18.png  is resized
1920 1080
EncryptionResult19.png  is resized
1920 1080
EncryptionResult20.png  is resized
1920 1080
EncryptionResult21.png  is resized
1920 1080
EncryptionResult22.png  is resized
1920 1080
Encryption